# Display a map showing road segment and corresponding SRI

## Importing libraries…

In [1]:
import pandas as pd
import os
import glob
import re
from pathlib import Path
import folium
from folium import plugins

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

## Organize Data

### Orgnize the SRI data

In [16]:
def SRI_orgnize(time_stamp):

    # use glob to get all the csv files in the folder
    SRI_csv_files = glob.glob(os.path.join('SRI/','*.csv'))

    df_list = []

    # loop over the list of csv files
    for f in SRI_csv_files:
        dict = {}
        regex = re.compile(r'\d+')
        segment_id =  [int(x) for x in regex.findall(f)][0]
        dict['segment_id'] = segment_id
        dict['time'] = time_stamp
        # read csv file
        globals()['segment_id_'+str(segment_id) + '_df']=pd.read_csv(f)
        # Convert the time to datetime64
        globals()['segment_id_'+str(segment_id) + '_df']['time'] = pd.to_datetime(globals()['segment_id_'+str(segment_id) + '_df'].time)
        # Filter data for a specified time
        dict['SRI']=globals()['segment_id_'+str(segment_id) + '_df'].loc[globals()['segment_id_'+str(segment_id) + '_df']['time'] == time_stamp,'SRI'].iloc[0]
        
        data = pd.read_json(f"https://data.cityofchicago.org/resource/sxs8-h27x.json?$where=time%20between%20%27{'2019-09-27T07:00:00'}%27%20and%20%27{'2019-09-28T07:00:00'}%27&segment_id={segment_id}")
        dict['start_latitude']=data.iloc[0]['start_latitude']
        dict['start_longitude']=data.iloc[0]['start_longitude']
        dict['end_latitude']=data.iloc[0]['end_latitude']
        dict['end_longitude']=data.iloc[0]['end_longitude']
        globals()['filtered_'+'segment_id_'+str(segment_id) + '_df']=pd.DataFrame(dict,index=[segment_id])

        df_list.append(globals()['filtered_'+'segment_id_'+str(segment_id) + '_df'])
    globals()[time_stamp+'SRI_df'] = pd.concat(df_list) #merge data into one dataset

In [17]:
SRI_orgnize('2019-09-28T07:00:00')
globals()['2019-09-28T07:00:00'+'SRI_df'].head()

,segment_id,time,SRI,start_latitude,start_longitude,end_latitude,end_longitude
1000,1000,2019-09-28T07:00:00,1.919436,42.011599,-87.728599,42.011720,-87.709057
1001,1001,2019-09-28T07:00:00,1.789435,42.011720,-87.709057,42.011976,-87.699747
1003,1003,2019-09-28T07:00:00,1.591629,42.012163,-87.690242,42.012105,-87.683067
100,100,2019-09-28T07:00:00,1.083526,41.735789,-87.662882,41.743076,-87.663068
1011,1011,2019-09-28T07:00:00,-0.001779,42.011854,-87.806576,42.011596,-87.816326


In [18]:
SRI_orgnize('2019-09-21T07:00:00')
globals()['2019-09-21T07:00:00'+'SRI_df'].head()

,segment_id,time,SRI,start_latitude,start_longitude,end_latitude,end_longitude
1000,1000,2019-09-21T07:00:00,1.919436,42.011599,-87.728599,42.011720,-87.709057
1001,1001,2019-09-21T07:00:00,1.789435,42.011720,-87.709057,42.011976,-87.699747
1003,1003,2019-09-21T07:00:00,1.677996,42.012163,-87.690242,42.012105,-87.683067
100,100,2019-09-21T07:00:00,-0.755735,41.735789,-87.662882,41.743076,-87.663068
1011,1011,2019-09-21T07:00:00,-0.001779,42.011854,-87.806576,42.011596,-87.816326


In [19]:
SRI_orgnize('2019-10-18T07:00:00')
globals()['2019-10-18T07:00:00'+'SRI_df'].head()

,segment_id,time,SRI,start_latitude,start_longitude,end_latitude,end_longitude
1000,1000,2019-10-18T07:00:00,1.410867,42.011599,-87.728599,42.011720,-87.709057
1001,1001,2019-10-18T07:00:00,1.916404,42.011720,-87.709057,42.011976,-87.699747
1003,1003,2019-10-18T07:00:00,1.448629,42.012163,-87.690242,42.012105,-87.683067
100,100,2019-10-18T07:00:00,1.599565,41.735789,-87.662882,41.743076,-87.663068
1011,1011,2019-10-18T07:00:00,-0.001779,42.011854,-87.806576,42.011596,-87.816326


In [49]:
def RRI_orgnize(time_stamp,station_id_list):
    df_list = []
    for station_id in station_id_list:
        dict = {}


        dict['station_id']= station_id
        dict['time'] = time_stamp
        globals()['station_id_'+str(station_id) + '_df']=pd.read_csv(Path('RRI/station_' + str(station_id) + '_RRI.csv'))
        dict['RRI']=globals()['station_id_'+str(station_id) + '_df'].loc[globals()['station_id_'+str(station_id) + '_df']['date'] == time_stamp,'RRI'].iloc[0]
        globals()['filtered_'+'station_id_'+str(station_id) + '_df']=pd.DataFrame(dict,index=[station_id])
        df_list.append(globals()['filtered_'+'station_id_'+str(station_id) + '_df'])

        
    globals()[time_stamp+'RRI_df'] = pd.concat(df_list) #merge data into one dataset

In [56]:
RRI_orgnize('2019-09-28',[40890,40820])
globals()['2019-09-28'+'RRI_df'].head()

,station_id,time,RRI
40890,40890,2019-09-28,8.468513
40820,40820,2019-09-28,-4.262072


In [57]:
RRI_orgnize('2019-09-21',[40890,40820])
globals()['2019-09-21'+'RRI_df'].head()

,station_id,time,RRI
40890,40890,2019-09-21,3.327505
40820,40820,2019-09-21,3.894958


In [51]:
RRI_orgnize('2019-10-18',[40890,40820])
globals()['2019-10-18'+'RRI_df'].head()

,station_id,time,RRI
40890,40890,2019-10-18,-6.365314
40820,40820,2019-10-18,1.028670


## Create a map centered on Chicago.

In [42]:
def SRI_gradient(middle,max,num):
    scale = 255 / (max-middle)

    if num > middle:
        return '#%02x%02x%02x' % (255 - int( ( num - middle ) * scale), 0, 0)
    else:
        return '#00FF00'


In [67]:
def visulization(time_stamp):    
    m = folium.Map(location=[41.8781, -87.6798], zoom_start=11,title='SRI on' + time_stamp)

    for index, row in globals()[time_stamp+'T07:00:00'+'SRI_df'].iterrows():
        start_lat = row['start_latitude']
        # print(start_lat)
        start_lon = row['start_longitude']
        end_lat = row['end_latitude']
        end_lon = row['end_longitude']
        SRI = row['SRI']
        points = []
        points.append([start_lat, start_lon])
        points.append([end_lat, end_lon])
        folium.PolyLine(points,color=SRI_gradient(2,10,SRI)).add_to(m)

    for index, row in globals()[time_stamp+'RRI_df'].iterrows():
        point = []
        if row['station_id'] == 40890:
            point.append([41.981127, -87.900876])
            RRI = row['RRI']
            folium.Marker((41.981127, -87.900876)).add_to(m)
        else:
            point.append([41.983507, -87.859388])
            RRI = row['RRI']
            folium.Marker((41.983507, -87.859388)).add_to(m)



    # Display the map.
    display(m)

    # Save map
    filepath = Path('map/station_' + str(time_stamp) + '.html')
    m.save(filepath)

visulization('2019-09-28')
visulization('2019-09-21')
visulization('2019-10-18')
